In [1]:
# standard python utilities
import os
from os.path import basename, dirname
import sys
import glob
import pandas as pd
import numpy as np
import calendar
import time

# standard python plotting utilities
import matplotlib as mpl
import matplotlib.pyplot as plt

# standard geospatial python utilities
import pyproj # for converting proj4string
import shapely
import geopandas as gpd
import rasterio

# mapping utilities
import contextily as ctx
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from mpl_toolkits.axes_grid1.anchored_artists import AnchoredSizeBar
import matplotlib.font_manager as fm

# modflow utility
import flopy
import flopy.utils.binaryfile as bf


In [2]:
## Set up directory referencing
# Package data
git_dir = os.path.dirname(os.path.dirname(os.getcwd()))
git_dir = os.getcwd()
while os.path.basename(git_dir) != 'CosumnesRiverRecharge':
    git_dir = os.path.dirname(git_dir)
gwfm_dir = '\\'.join(str.split(git_dir,'\\')[0:3])+ '/Box/research_cosumnes/GWFlowModel'
print(git_dir, gwfm_dir)

C:\Users\ajcalder\Documents\GitHub\CosumnesRiverRecharge C:\Users\ajcalder/Box/research_cosumnes/GWFlowModel


In [3]:
# set box directory for output figures and data
box_dir = gwfm_dir+'/Levee_setback/levee_setback_distance_analysis/'

# tprogs_id = '' # original tprogs with conditioning data in output tsim
tprogs_id = '_no_conditioning'

data_dir = box_dir+ tprogs_id+'/data_output/'
fig_dir = box_dir+tprogs_id+'/figures/'
#
chan_dir = box_dir+'channel_data/'
gis_dir = chan_dir+'GIS/'

check_stream_loc = False
# fig_dir = box_dir+'north_shifted_stream_distance_analysis/figures/'


In [4]:
# dem data for cropping above land surface
dem_data = np.loadtxt(gwfm_dir+'/DIS_data/dem_52_9_200m_linear.tsv')

In [5]:
grid_p = gpd.read_file(gwfm_dir+'/DIS_data/grid/grid.shp')


C:\Users\ajcalder\Anaconda3\envs\geosp\lib\site-packages\geopandas\_vectorized.py:142: DeprecationWarning: An exception was ignored while fetching the attribute `__array_interface__` from an object of type 'Polygon'.  With the exception of `AttributeError` NumPy will always raise this exception in the future.  Raise this deprecation warning to see the original exception. (Warning added NumPy 1.21)
  aout[:] = out
C:\Users\ajcalder\Anaconda3\envs\geosp\lib\site-packages\geopandas\_vectorized.py:142: DeprecationWarning: An exception was ignored while fetching the attribute `__array_interface__` from an object of type 'Polygon'.  With the exception of `AttributeError` NumPy will always raise this exception in the future.  Raise this deprecation warning to see the original exception. (Warning added NumPy 1.21)
  aout[:] = out
C:\Users\ajcalder\Anaconda3\envs\geosp\lib\site-packages\geopandas\_vectorized.py:142: DeprecationWarning: An exception was ignored while fetching the attribute `__ar

C:\Users\ajcalder\Anaconda3\envs\geosp\lib\site-packages\geopandas\_vectorized.py:142: DeprecationWarning: An exception was ignored while fetching the attribute `__array_interface__` from an object of type 'Polygon'.  With the exception of `AttributeError` NumPy will always raise this exception in the future.  Raise this deprecation warning to see the original exception. (Warning added NumPy 1.21)
  aout[:] = out
C:\Users\ajcalder\Anaconda3\envs\geosp\lib\site-packages\geopandas\_vectorized.py:142: DeprecationWarning: An exception was ignored while fetching the attribute `__array_interface__` from an object of type 'Polygon'.  With the exception of `AttributeError` NumPy will always raise this exception in the future.  Raise this deprecation warning to see the original exception. (Warning added NumPy 1.21)
  aout[:] = out
C:\Users\ajcalder\Anaconda3\envs\geosp\lib\site-packages\geopandas\_vectorized.py:142: DeprecationWarning: An exception was ignored while fetching the attribute `__ar

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [8]:
import numpy as np

import pyvista as pv
from pyvista import examples

In [ ]:
pv.set_jupyter_backend('pythreejs')

In [196]:
data = examples.load_channels()
channels = data.threshold([0.9, 1.1])

C:\Users\ajcalder\Anaconda3\envs\geosp\lib\site-packages\pyvista\core\filters\data_set.py:872: DeprecationWarning: Call to deprecated method ThresholdBetween. (Use 'SetLowerThreshold', 'SetUpperThreshold' and 'SetThresholdFunction' instead.) -- Deprecated since version 9.1.0.
  alg.ThresholdBetween(value[0], value[1])


In [194]:
data

UniformGrid (0x271a3f40ac8)
  N Cells:	6250000
  N Points:	6363101
  X Bounds:	0.000e+00, 2.500e+02
  Y Bounds:	0.000e+00, 2.500e+02
  Z Bounds:	0.000e+00, 1.000e+02
  Dimensions:	251, 251, 101
  Spacing:	1.000e+00, 1.000e+00, 1.000e+00
  N Arrays:	1

In [198]:
channels

UnstructuredGrid (0x271a48d44c8)
  N Cells:	684869
  N Points:	868523
  X Bounds:	0.000e+00, 2.500e+02
  Y Bounds:	0.000e+00, 2.500e+02
  Z Bounds:	0.000e+00, 1.000e+02
  N Arrays:	1

In [127]:
channels.plot(scalars='facies',cmap='viridis')

C:\Users\ajcalder\Anaconda3\envs\geosp\lib\site-packages\pyvista\core\pointset.py:483: DeprecationWarning: ``is_all_triangles`` is now property as of 0.32.0 and does not need ()
  'does not need ()', DeprecationWarning)


Renderer(camera=PerspectiveCamera(aspect=1.3333333333333333, children=(DirectionalLight(color='#fefefe', inten…

In [66]:
mf_tprogs_dir = gwfm_dir+'/UPW_data/tprogs_final' + tprogs_id+'/'
tprogs_files = glob.glob(mf_tprogs_dir+'*')
# tprogs_files

In [67]:
t=0
tprogs_line = np.loadtxt(tprogs_files[t])
tprogs_arr = np.reshape(tprogs_line, (320, 100,230))

# K, Sy, Ss= int_to_param(tprogs_arr)


In [133]:
## old way, doesn't create a pretty grid directly
# x = grid_p.geometry.centroid.x.values
# y = grid_p.geometry.centroid.y.values
# #
# z = np.arange(-80, 80, 0.5)
# zz = np.repeat(z, x.shape)
# xx = np.tile(x, z.shape)
# yy = np.tile(y, z.shape)

# points = np.c_[xx, yy, zz]

# mesh = pv.StructuredGrid()
# # Set the coordinates from the numpy array
# mesh.points = points
# # set the dimensions
# mesh.dimensions = [100, 230, 320]

# add tprogs facies to mesh
# arr_in = np.moveaxis(tprogs_arr,0,2).flatten(order='F').astype(int)
# mesh.add_field_data(arr_in, name='facies')


In [219]:
arr_in.shape
grid

UniformGrid (0x271a48eab28)
  N Cells:	7360000
  N Points:	7489251
  X Bounds:	6.455e+05, 6.655e+05
  Y Bounds:	4.228e+06, 4.274e+06
  Z Bounds:	-8.000e+01, 8.000e+01
  Dimensions:	101, 231, 321
  Spacing:	2.000e+02, 2.000e+02, 5.000e-01
  N Arrays:	1

In [218]:
grid = pv.UniformGrid()

# Set the grid dimensions: shape because we want to inject our values on the
#   POINT data
# I have to add 1 to each dimension to have it be built on the cells
grid.dimensions = [101, 231, 321]

grid.origin = (645500.0, 4227700.0, -80) # bottom left corner of the dataset
grid.spacing = (200,200,0.5)

arr_in = np.moveaxis(tprogs_arr,0,2).flatten(order='F').astype(int)
# grid.point_data["facies"] = arr_in
grid.cell_data["facies"] = arr_in

In [220]:
# and then inspect it!
# mesh.plot(scalars="facies", cmap='viridis', show_edges=False, show_grid=True) #, cpos="xy"
# plotter.set_scale(1, 1, 10)

plotter = pv.Plotter(notebook=False)
plotter.add_mesh(grid, scalars="facies", cmap='viridis')

plotter.set_scale(1, 1, 20)
plotter.show()

# plotting looks the same when built on cells instead of points

The mesh is looking pretty close. The side view looks correct for part of it but there is a weird extension that is continuous, maybe an issue with the way I repeated data to make x,y,z

If value is a single value, when invert is True cells are kept when their values are below parameter "value". When invert is False cells are kept when their value is above the threshold "value". Default is False: yielding above the threshold "value".


In [221]:
# filter mesh by facies (1,2 gravel and sand)
coarse = grid.threshold(value = [0.9, 2.1], scalars='facies') #, preference='cell'


C:\Users\ajcalder\Anaconda3\envs\geosp\lib\site-packages\pyvista\core\filters\data_set.py:872: DeprecationWarning: Call to deprecated method ThresholdBetween. (Use 'SetLowerThreshold', 'SetUpperThreshold' and 'SetThresholdFunction' instead.) -- Deprecated since version 9.1.0.
  alg.ThresholdBetween(value[0], value[1])


In [158]:
# very long process
# bodies = coarse.split_bodies()
# # Now remove all bodies with a small volume
# for key in bodies.keys():
#     b = bodies[key]
#     vol = b.volume
#     if vol < 1000.0:
#         del bodies[key]
#         continue
#     # Now lets add a volume array to all blocks
#     b.cell_data["TOTAL VOLUME"] = np.full(b.n_cells, vol)

In [225]:
largest = coarse.connectivity(largest=True)


In [226]:
# switching from points to cells makes the threshold analysis work

# coarse.plot()
# conn = mesh.connectivity(largest=False)

# conn.plot()
plotter = pv.Plotter(notebook=False)
plotter.add_mesh(largest, scalars="facies", cmap='viridis')
# plotter.add_mesh(coarse, scalars="facies", cmap='viridis')

plotter.set_scale(1, 1, 50)
plotter.show()
